
<h2>Recomendador de Canciones del Top50 de Spotify con K-means</>

Para quienes no estén muy familiarizados con el término, un algoritmo recomendador es un tipo de inteligencia artificial que se encarga de hacer recomendaciones personalizadas basadas en los gustos y preferencias de cada usuario. En nuestro caso, hemos utilizado un algoritmo recomendador para hacer recomendaciones de canciones en Spotify, basándonos en los géneros musicales preferidos y no preferidos de un usuario determinado.

Para llevar a cabo este proyecto, hemos utilizado Python y la biblioteca Spotipy, que nos permite acceder a la API de Spotify y obtener información sobre canciones, artistas, playlists, etc. También hemos usado diversas técnicas de análisis de datos y aprendizaje automático, como la extracción de características de audio de las canciones, la normalización de datos y el uso de un modelo de clasificación basado en vecinos cercanos.

La utilidad de los algoritmos recomendadores es innegable, ya que nos permiten descubrir nuevas canciones, películas, libros, productos, etc. que de otro modo quizás nunca hubiéramos conocido. Además, a medida que la cantidad de información disponible en línea sigue creciendo exponencialmente, los algoritmos recomendadores se vuelven cada vez más relevantes para ayudarnos a encontrar lo que estamos buscando.

Como usuario habitual de spotify siempre busco descubrir géneros nuevos, artistas desconocidos y música rara que a pena se suele colar en las listas de los temas más virales. Soy consciente de que la lista de top 50 tanto global como la de España varía además bastante a menudo. Por ello y de cara al veranito he decidido hacer este recomendador para que, con base en los gustos personales del usuario que se le da al inicio del cuaderno, el algoritmo sea capaz de darnos las canciones de ambos top que más nos pueden gustar.

Por tanto, si eres usuario de Spotify y quieres descubrir nuevas canciones basadas en tus gustos musicales no dudes en echarle un vistazo a este recomendador :)


In [ ]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '7d0bf6eb0e8a4145ab4cb73c46f16422'
client_secret = '0608f4cca00d415c846558397f79b0fd'
nombre_usuario = 'alexmor_22'  # Rellenar con el nombre de usuario del que se quiera sacar informacion
redirect_uri = 'http://localhost:8889/callback'

# Incluir los scopes 'user-library-read', 'user-top-read' y 'playlist-read-private'
scope = 'user-library-read user-top-read playlist-read-private'

auth_manager = SpotifyOAuth(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope,
                            username=nombre_usuario)

sp = spotipy.Spotify(auth_manager=auth_manager)


In [ ]:

def get_top_artists(sp, limit=100, time_range='medium_term'):
    top_artists = sp.current_user_top_artists(limit=limit, time_range=time_range)
    return top_artists['items']


top_artists = get_top_artists(sp)
print(top_artists)


def get_genres_from_artists(artists):
    genres = []
    for artist in artists:
        genres.extend(artist['genres'])
    return genres


top_genres = get_genres_from_artists(top_artists)
print(top_genres)
from collections import Counter


def get_top_n_genres(genres, n=5):
    counter = Counter(genres)
    return [item[0] for item in counter.most_common(n)]


top_5_genres = get_top_n_genres(top_genres)
print(top_5_genres)
generos_indiferentes = ['latino', 'funk', 'disco', 'Electro', 'techno', 'house', ]
generos_no_gustados = ['K-Pop', 'latino', 'trance', 'dubstep', 'reggaeton', 'ska']


def get_tracks_by_genres(sp, genres, limit=50):
    track_data = []
    for genre in genres:
        results = sp.search(q=f'genre:"{genre}"', type='track', limit=limit)
        tracks = results['tracks']['items']

        for track in tracks:
            track_data.append({
                'id': track['id'],
                'name': track['name'],
                'artists': [artist['name'] for artist in track['artists']],
                'album': track['album']['name'],
                'popularity': track['popularity']
            })
    return track_data


def get_audio_features(track_ids):
    audio_features = []
    for track_id in track_ids:
        features = sp.audio_features(track_id)
        audio_features.append(features)
    return audio_features


In [ ]:

preferred_genres_tracks = get_tracks_by_genres(sp, top_5_genres)
indifferent_genres_tracks = get_tracks_by_genres(sp, generos_indiferentes)
disliked_genres_tracks = get_tracks_by_genres(sp, generos_no_gustados)

preferred_genres_df = pd.DataFrame(preferred_genres_tracks)
indifferent_genres_df = pd.DataFrame(indifferent_genres_tracks)
disliked_genres_df = pd.DataFrame(disliked_genres_tracks)

preferred_genres_df['preferencia'] = 1
indifferent_genres_df['preferencia'] = 0
disliked_genres_df['preferencia'] = -1

preferred_genres_df.to_csv('generos_preferidos.csv', index=False)
indifferent_genres_df.to_csv('generos_indiferentes.csv', index=False)
disliked_genres_df.to_csv('generos_no_gustados.csv', index=False)


In [ ]:

def add_audio_features_to_df(df):
    audio_features_list = []
    for track_id in df['id']:
        audio_features = sp.audio_features(track_id)[0]
        if audio_features:
            audio_features_list.append(audio_features)
        else:
            audio_features_list.append(None)

    audio_features_df = pd.DataFrame(audio_features_list,
                                     columns=['id', 'danceability', 'energy', 'acousticness', 'instrumentalness',
                                              'tempo', 'valence', 'duration_ms', 'time_signature', 'key'])
    df = df.merge(audio_features_df, on='id')
    return df


preferred_genres_df = add_audio_features_to_df(preferred_genres_df)
indifferent_genres_df = add_audio_features_to_df(indifferent_genres_df)
disliked_genres_df = add_audio_features_to_df(disliked_genres_df)

datos_totales = pd.concat([preferred_genres_df, indifferent_genres_df, disliked_genres_df], ignore_index=True)

print(datos_totales)

print(datos_totales.columns)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def normalize_audio_features(df):
    scaler = MinMaxScaler()
    features = ['danceability', 'energy', 'acousticness', 'instrumentalness', 'tempo', 'valence', 'duration_ms',
                'time_signature', 'key']
    df[features] = scaler.fit_transform(df[features])
    return df


datos_totales = normalize_audio_features(datos_totales)

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Seleccione las características de audio y la columna de preferencia
caracteristicas = ['danceability', 'energy', 'acousticness', 'instrumentalness', 'tempo', 'valence', 'duration_ms',
                   'time_signature', 'key']
X = datos_totales[caracteristicas]
y = datos_totales['preferencia']

# Divide los datos en conjuntos de entrenamiento y prueba
X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(X, y, test_size=0.2, random_state=42)

k = 3
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_entrenamiento, y_entrenamiento)
y_pred = knn.predict(X_prueba)

print("Matriz de confusión:\n", confusion_matrix(y_prueba, y_pred))
print("\nReporte de clasificación:\n", classification_report(y_prueba, y_pred))
print("Precisión:", accuracy_score(y_prueba, y_pred))


In [ ]:

def obtener_id_playlist(nombre_playlist):
    resultados = sp.search(q=nombre_playlist, type='playlist', limit=1)
    playlist_id = resultados['playlists']['items'][0]['id']
    return playlist_id


def obtener_canciones_playlist(playlist_id):
    resultados = sp.playlist_tracks(playlist_id)
    canciones = resultados['items']
    return canciones


# Obtenemos las ID de las playlists de virales de España y del mundo
id_virales_españa = obtener_id_playlist('Viral 50 - España')
id_virales_global = obtener_id_playlist('Viral 50 - Global')

# Obtenemos las canciones de cada playlist
canciones_virales_españa = obtener_canciones_playlist(id_virales_españa)
canciones_virales_global = obtener_canciones_playlist(id_virales_global)

# Combinamos ambas listas de canciones
canciones_virales = canciones_virales_españa + canciones_virales_global

print(canciones_virales)


In [ ]:
def extraer_informacion_canciones(canciones):
    info_canciones = []
    for cancion in canciones:
        track = cancion['track']
        info_canciones.append({
            'id': track['id'],
            'name': track['name'],
            'artists': [artist['name'] for artist in track['artists']],
            'album': track['album']['name'],
            'popularity': track['popularity']
        })
    return info_canciones


# Extraemos la información relevante de las canciones virales
info_canciones_virales = extraer_informacion_canciones(canciones_virales)

# Creamos un DataFrame con la información de las canciones virales
canciones_virales_df = pd.DataFrame(info_canciones_virales)

print(canciones_virales_df)
print(canciones_virales_df.columns)

In [ ]:
# Añadimos las características de audio al DataFrame
canciones_virales_df = add_audio_features_to_df(canciones_virales_df)

# Normalizamos las características de audio
canciones_virales_df = normalize_audio_features(canciones_virales_df)

# Predecimos la preferencia del usuario utilizando el modelo entrenado
X_virales = canciones_virales_df[
    ['danceability', 'energy', 'acousticness', 'instrumentalness', 'tempo', 'valence', 'duration_ms', 'time_signature',
     'key']]
y_pred_virales = knn.predict(X_virales)

# Filtramos las canciones recomendadas (aquellas con una preferencia de 1)
canciones_recomendadas = canciones_virales_df[y_pred_virales == 1]

# Imprimimos las canciones recomendadas
print("Canciones recomendadas:")
print(canciones_recomendadas)